In [1]:
# Установим Spark:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
# Установим переменные среды:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [4]:
pip install pyspark==3.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 21.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767581 sha256=61db703052ff439ae8adcc21b494bb05cde89edbd898b3b5fa3b7f3e6e92de1a
  Stored in directory: /root/.cache/pip/wheels/a0/3f/72/8efd988f9ae041f051c75e6834cd92dd6d13a726e206e8b6f3
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

In [6]:
sc = spark.sparkContext

In [7]:
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/bachelor/data/programming-languages.csv

--2024-04-20 10:32:38--  https://git.ai.ssau.ru/tk/big_data/raw/branch/bachelor/data/programming-languages.csv
Resolving git.ai.ssau.ru (git.ai.ssau.ru)... 91.222.131.161
Connecting to git.ai.ssau.ru (git.ai.ssau.ru)|91.222.131.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40269 (39K) [text/plain]
Saving to: ‘programming-languages.csv’

programming-languag 100%[===================>]  39.33K   115KB/s    in 0.3s    

2024-04-20 10:32:40 (115 KB/s) - ‘programming-languages.csv’ saved [40269/40269]



In [8]:
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/bachelor/data/posts_sample.xml

--2024-04-20 10:32:40--  https://git.ai.ssau.ru/tk/big_data/raw/branch/bachelor/data/posts_sample.xml
Resolving git.ai.ssau.ru (git.ai.ssau.ru)... 91.222.131.161
Connecting to git.ai.ssau.ru (git.ai.ssau.ru)|91.222.131.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74162295 (71M) [text/plain]
Saving to: ‘posts_sample.xml’

posts_sample.xml    100%[===================>]  70.73M   457KB/s    in 3m 37s  

2024-04-20 10:36:18 (334 KB/s) - ‘posts_sample.xml’ saved [74162295/74162295]



In [9]:
!ls

posts_sample.xml	   sample_data		      spark-3.1.1-bin-hadoop3.2.tgz
programming-languages.csv  spark-3.1.1-bin-hadoop3.2


In [10]:
# Загрузим и изучим данные о языках программирования:
languagesRaw = sc.textFile('programming-languages.csv')
languagesRaw.take(5)

['name,wikipedia_url',
 'A# .NET,https://en.wikipedia.org/wiki/A_Sharp_(.NET)',
 'A# (Axiom),https://en.wikipedia.org/wiki/A_Sharp_(Axiom)',
 'A-0 System,https://en.wikipedia.org/wiki/A-0_System',
 'A+,https://en.wikipedia.org/wiki/A%2B_(programming_language)']

In [11]:
# Удалим лишнюю первую строку из данных:
firstRaw = languagesRaw.first()
languagesRaw = languagesRaw.filter(lambda raw: raw != firstRaw)
languagesRaw.take(5)

['A# .NET,https://en.wikipedia.org/wiki/A_Sharp_(.NET)',
 'A# (Axiom),https://en.wikipedia.org/wiki/A_Sharp_(Axiom)',
 'A-0 System,https://en.wikipedia.org/wiki/A-0_System',
 'A+,https://en.wikipedia.org/wiki/A%2B_(programming_language)',
 'A++,https://en.wikipedia.org/wiki/A%2B%2B']

In [12]:
# Превратим данные просто в список языков программирования:
languages = languagesRaw.map(lambda raw: raw.split(',', -1)[0].lower()) \
                        .collect()
print(languages)

['a# .net', 'a# (axiom)', 'a-0 system', 'a+', 'a++', 'abap', 'abc', 'abc algol', 'abset', 'absys', 'acc', 'accent', 'ace dasl', 'acl2', 'act-iii', 'action!', 'actionscript', 'ada', 'adenine', 'agda', 'agilent vee', 'agora', 'aimms', 'alef', 'alf', 'algol 58', 'algol 60', 'algol 68', 'algol w', 'alice', 'alma-0', 'ambienttalk', 'amiga e', 'amos', 'ampl', 'apex (salesforce.com)', 'apl', "app inventor for android's visual block language", 'applescript', 'arc', 'arexx', 'argus', 'aspectj', 'assembly language', 'ats', 'ateji px', 'autohotkey', 'autocoder', 'autoit', 'autolisp / visual lisp', 'averest', 'awk', 'axum', 'b', 'babbage', 'bash', 'basic', 'bc', 'bcpl', 'beanshell', 'batch (windows/dos)', 'bertrand', 'beta', 'bigwig', 'bistro', 'bitc', 'bliss', 'blockly', 'bloop', 'blue', 'boo', 'boomerang', 'bourne shell (including', 'bash and', 'ksh )', 'brew', 'bpel', 'c', 'c--', 'c++ – iso/iec 14882', 'c# – iso/iec 23270', 'c/al', 'caché objectscript', 'c shell', 'caml', 'cayenne', 'cduce', 'c

In [13]:
# Загрузим и изучим данные с постами stackoverflow:
postsRaw = sc.textFile('posts_sample.xml')
postsRaw.take(5)

['<?xml version="1.0" encoding="utf-8"?>',
 '<posts>',
 '  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="630" ViewCount="42817" Body="&lt;p&gt;I want to use a track-bar to change a form\'s opacity.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;Cannot implicitly convert type &lt;code&gt;\'decimal\'&lt;/code&gt; to &lt;code&gt;\'double\'&lt;/code&gt;&lt;/p&gt;&#xA;&lt;/blockquote&gt;&#xA;&#xA;&lt;p&gt;I tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt; but then the control doesn\'t work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserId="8" LastEditorUserId="3641067" LastEditorDisplayName="Rich B" LastEditDate="2019-07-19T0

In [14]:
postsRaw.count()

46009

In [15]:
# Удалим первые две строки и последнюю, чтобы остались только посты:
postsCount = postsRaw.count()
posts = postsRaw.zipWithIndex() \
                .filter(lambda x: x[1] > 1 and x[1] < postsCount - 1) \
                .map(lambda x: x[0])
posts.take(5)

['  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="630" ViewCount="42817" Body="&lt;p&gt;I want to use a track-bar to change a form\'s opacity.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;Cannot implicitly convert type &lt;code&gt;\'decimal\'&lt;/code&gt; to &lt;code&gt;\'double\'&lt;/code&gt;&lt;/p&gt;&#xA;&lt;/blockquote&gt;&#xA;&#xA;&lt;p&gt;I tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt; but then the control doesn\'t work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserId="8" LastEditorUserId="3641067" LastEditorDisplayName="Rich B" LastEditDate="2019-07-19T01:39:54.173" LastActivityDate="2019-07-19T01:39:54.173"

In [16]:
posts.count()

46006

**Теперь попробуем раcпарсить этот xml файл.**

In [17]:
# Превратим каждую строку из posts в объект xml, чтобы вытащить из них нужные поля (дату создания и теги):
import xml.etree.ElementTree as ET

def extractCreationDateAndTags(e):
    [creationDate, tags] = e.attrib.get('CreationDate'), e.attrib.get('Tags')
    return creationDate, tags

posts_xml = posts.map(lambda post: ET.fromstring(post)) \
                 .map(lambda x: extractCreationDateAndTags(x)) \
                 .filter(lambda x: x[0] is not None and x[1] is not None)
posts_xml.take(10)

[('2008-07-31T21:42:52.667',
  '<c#><floating-point><type-conversion><double><decimal>'),
 ('2008-07-31T22:08:08.620', '<html><css><internet-explorer-7>'),
 ('2008-07-31T23:40:59.743', '<c#><.net><datetime>'),
 ('2008-07-31T23:55:37.967',
  '<c#><datetime><time><datediff><relative-time-span>'),
 ('2008-08-01T00:42:38.903',
  '<html><browser><timezone><user-agent><timezone-offset>'),
 ('2008-08-01T00:59:11.177', '<.net><math>'),
 ('2010-09-22T10:33:21.790', '<c++><character-encoding>'),
 ('2010-09-23T06:47:28.920', '<sharepoint><infopath>'),
 ('2010-09-23T08:53:14.017', '<iphone><app-store><in-app-purchase>'),
 ('2010-09-23T11:47:00.833', '<symfony1><schema><doctrine><fixtures>')]

In [18]:
posts_xml.count()

18055

In [19]:
# Вытащим из даты год и разобъем строку тегов на массив отдельных тегов:
def parseCreationDateAndTags(e):
    [creationDate, tags] = e
    tags = tags[:-1].replace('<', '').split('>')
    year = creationDate[:4]
    return (year, tags)

postYearTags = posts_xml.map(lambda e: parseCreationDateAndTags(e))
postYearTags.take(10)

[('2008', ['c#', 'floating-point', 'type-conversion', 'double', 'decimal']),
 ('2008', ['html', 'css', 'internet-explorer-7']),
 ('2008', ['c#', '.net', 'datetime']),
 ('2008', ['c#', 'datetime', 'time', 'datediff', 'relative-time-span']),
 ('2008', ['html', 'browser', 'timezone', 'user-agent', 'timezone-offset']),
 ('2008', ['.net', 'math']),
 ('2010', ['c++', 'character-encoding']),
 ('2010', ['sharepoint', 'infopath']),
 ('2010', ['iphone', 'app-store', 'in-app-purchase']),
 ('2010', ['symfony1', 'schema', 'doctrine', 'fixtures'])]

In [20]:
postYearTags.count()

18055

In [21]:
# Теперь сформируем пары год-тег:
def flatPostYearTags(e):
    return [(e[0], y) for y in e[1]]

yearTags = postYearTags.flatMap(flatPostYearTags)
yearTags.take(10)

[('2008', 'c#'),
 ('2008', 'floating-point'),
 ('2008', 'type-conversion'),
 ('2008', 'double'),
 ('2008', 'decimal'),
 ('2008', 'html'),
 ('2008', 'css'),
 ('2008', 'internet-explorer-7'),
 ('2008', 'c#'),
 ('2008', '.net')]

In [22]:
yearTags.count()

53297

In [23]:
# Отфильтруем только те теги, которые содержатся в списке языков программирования:
yearLanguageTags = yearTags.filter(lambda record: record[1] in languages)
yearLanguageTags.take(10)

[('2010', 'java'),
 ('2010', 'php'),
 ('2010', 'ruby'),
 ('2010', 'c'),
 ('2010', 'php'),
 ('2010', 'python'),
 ('2010', 'javascript'),
 ('2010', 'applescript'),
 ('2010', 'php'),
 ('2010', 'php')]

In [24]:
yearLanguageTags.count()

8204

In [25]:
# Сгенерируем список годов, за которые нам нужно сформировать таблицы отчета:
years = list(map(str, [year for year in range(2010, 2020)]))
print(years)

['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']


In [26]:
# Для каждого года из списка посчитаем количества упоминаний каждого языка в постах на stackoverflow:
yearsTagCounts = map(lambda year: yearLanguageTags.filter(lambda record: record[0] == year) \
                                                         .map(lambda x: (x[1], 1)) \
                                                         .reduceByKey(lambda a, b: a + b) \
                                                         .map(lambda y: (year, y[0], y[1])), years)
yearsTagCounts = list(yearsTagCounts)
print(yearsTagCounts)

[PythonRDD[62] at RDD at PythonRDD.scala:53, PythonRDD[63] at RDD at PythonRDD.scala:53, PythonRDD[64] at RDD at PythonRDD.scala:53, PythonRDD[65] at RDD at PythonRDD.scala:53, PythonRDD[66] at RDD at PythonRDD.scala:53, PythonRDD[67] at RDD at PythonRDD.scala:53, PythonRDD[68] at RDD at PythonRDD.scala:53, PythonRDD[69] at RDD at PythonRDD.scala:53, PythonRDD[70] at RDD at PythonRDD.scala:53, PythonRDD[71] at RDD at PythonRDD.scala:53]


In [27]:
yearsTagCounts[0].take(10)

[('2010', 'php', 46),
 ('2010', 'python', 26),
 ('2010', 'javascript', 44),
 ('2010', 'applescript', 3),
 ('2010', 'sed', 2),
 ('2010', 'xpath', 1),
 ('2010', 'java', 52),
 ('2010', 'objective-c', 23),
 ('2010', 'basic', 1),
 ('2010', 'f#', 2)]

In [28]:
# Для каждого года отфильтруем только топ-10 самых популярных языков программирования:
topYearsTagCounts = list(map(lambda rdd: rdd.sortBy(lambda raw: raw[2], False).top(10, lambda x: x[2]), yearsTagCounts))
print(topYearsTagCounts)

[[('2010', 'java', 52), ('2010', 'php', 46), ('2010', 'javascript', 44), ('2010', 'python', 26), ('2010', 'objective-c', 23), ('2010', 'c', 20), ('2010', 'ruby', 12), ('2010', 'delphi', 8), ('2010', 'applescript', 3), ('2010', 'r', 3)], [('2011', 'php', 102), ('2011', 'java', 93), ('2011', 'javascript', 83), ('2011', 'python', 37), ('2011', 'objective-c', 34), ('2011', 'c', 24), ('2011', 'ruby', 20), ('2011', 'perl', 9), ('2011', 'delphi', 8), ('2011', 'bash', 7)], [('2012', 'php', 154), ('2012', 'javascript', 132), ('2012', 'java', 124), ('2012', 'python', 69), ('2012', 'objective-c', 45), ('2012', 'ruby', 27), ('2012', 'c', 27), ('2012', 'bash', 10), ('2012', 'r', 9), ('2012', 'xpath', 6)], [('2013', 'javascript', 198), ('2013', 'php', 198), ('2013', 'java', 194), ('2013', 'python', 90), ('2013', 'objective-c', 40), ('2013', 'c', 36), ('2013', 'ruby', 32), ('2013', 'r', 26), ('2013', 'bash', 12), ('2013', 'scala', 11)], [('2014', 'javascript', 239), ('2014', 'java', 228), ('2014', 'p

In [29]:
# Сгладим полученный список:
from functools import reduce

finalReport = reduce(lambda x, y: x + y, topYearsTagCounts)
print(finalReport)

[('2010', 'java', 52), ('2010', 'php', 46), ('2010', 'javascript', 44), ('2010', 'python', 26), ('2010', 'objective-c', 23), ('2010', 'c', 20), ('2010', 'ruby', 12), ('2010', 'delphi', 8), ('2010', 'applescript', 3), ('2010', 'r', 3), ('2011', 'php', 102), ('2011', 'java', 93), ('2011', 'javascript', 83), ('2011', 'python', 37), ('2011', 'objective-c', 34), ('2011', 'c', 24), ('2011', 'ruby', 20), ('2011', 'perl', 9), ('2011', 'delphi', 8), ('2011', 'bash', 7), ('2012', 'php', 154), ('2012', 'javascript', 132), ('2012', 'java', 124), ('2012', 'python', 69), ('2012', 'objective-c', 45), ('2012', 'ruby', 27), ('2012', 'c', 27), ('2012', 'bash', 10), ('2012', 'r', 9), ('2012', 'xpath', 6), ('2013', 'javascript', 198), ('2013', 'php', 198), ('2013', 'java', 194), ('2013', 'python', 90), ('2013', 'objective-c', 40), ('2013', 'c', 36), ('2013', 'ruby', 32), ('2013', 'r', 26), ('2013', 'bash', 12), ('2013', 'scala', 11), ('2014', 'javascript', 239), ('2014', 'java', 228), ('2014', 'php', 167)

In [30]:
# Преобразуем полученный список в Data Frame:
finalDataFrame = sc.parallelize(finalReport).toDF(['Year', 'Language', 'Count'])
finalDataFrame.show(10 * len(years))

+----+-----------+-----+
|Year|   Language|Count|
+----+-----------+-----+
|2010|       java|   52|
|2010|        php|   46|
|2010| javascript|   44|
|2010|     python|   26|
|2010|objective-c|   23|
|2010|          c|   20|
|2010|       ruby|   12|
|2010|     delphi|    8|
|2010|applescript|    3|
|2010|          r|    3|
|2011|        php|  102|
|2011|       java|   93|
|2011| javascript|   83|
|2011|     python|   37|
|2011|objective-c|   34|
|2011|          c|   24|
|2011|       ruby|   20|
|2011|       perl|    9|
|2011|     delphi|    8|
|2011|       bash|    7|
|2012|        php|  154|
|2012| javascript|  132|
|2012|       java|  124|
|2012|     python|   69|
|2012|objective-c|   45|
|2012|       ruby|   27|
|2012|          c|   27|
|2012|       bash|   10|
|2012|          r|    9|
|2012|      xpath|    6|
|2013| javascript|  198|
|2013|        php|  198|
|2013|       java|  194|
|2013|     python|   90|
|2013|objective-c|   40|
|2013|          c|   36|
|2013|       ruby|   32|


In [31]:
# Сохраним полученный отчет в формате Apache Parquet:
finalDataFrame.write.parquet("/content/top_10_languages_by_year")